In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))
from compute_flops import estimate_max_training_steps

max_steps = estimate_max_training_steps(
    data_path="../lotka_volterra_data.h5",
    lora_rank=2,
    input_fraction=1,
    batch_size=4,
    flop_budget=1e17,
    train_series_count=700,
    eval_series_count=300,
    context_length=512
)


/Users/raunaqrai/miniconda_x86_64/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Users/raunaqrai/miniconda_x86_64/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



 Avg token count: 512
 FLOPs per training step (batch): 6.75e+12
 Evaluation FLOPs: 1.69e+14
 Remaining budget for training: 9.98e+16
 Max training steps allowed: 14781


In [5]:
import sys
import os
sys.path.append(os.path.abspath("../src"))
from compute_flops import compute_flops

compute_flops(
    data_path="../lotka_volterra_data.h5",
    input_fraction=1,
    lora_rank=8,
    batch_size=4,
    training_steps=600,
    flop_budget=1e17,
    train_series_count=700,
    eval_series_count=300,
    context_length=512
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



 Context length used for FLOP estimation: 512 tokens

  Total Training FLOPs: 4.06e+15
  Total Evaluation FLOPs: 1.69e+14
  Total Combined FLOPs: 4.23e+15
  Percentage of FLOP budget used: 4.22516%


In [4]:
import sys
import os
sys.path.append(os.path.abspath("../src"))
from compute_flops import compute_flops

compute_flops(
    data_path="../lotka_volterra_data.h5",
    input_fraction=1,
    lora_rank=8,
    batch_size=4,
    training_steps=5000,
    flop_budget=1e17,
    train_series_count=700,
    eval_series_count=300,
    context_length=768
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



 Context length used for FLOP estimation: 768 tokens

  Total Training FLOPs: 5.17e+16
  Total Evaluation FLOPs: 2.59e+14
  Total Combined FLOPs: 5.20e+16
  Percentage of FLOP budget used: 51.98719%


- Start with 1000 time series
- Split immediately → 700 for training, 300 for evaluation
- Tokenise the 700 training series into sequences
- Train the model on this
- Evaluate the model on the validation/test split (the 300)